In [ ]:
# Hyperparameters
batch_size = 256
learning_rate = 1e-5

# Other constants
input_size = 28*28
num_classes = 26

# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
# Imports
import torch
import torchvision
import string
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [ ]:
# Azure ML Imports
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'ASL-DeepLearning-hyperameter'

experiment=Experiment(ws, experiment_name)


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://www.kaggle.com/datamunge/sign-language-mnist"

from azureml.data.dataset_factory import TabularDatasetFactory
datastore_path = "https://github.com/emanbuc/ASL-Recognition-Deep-Learning/raw/main/datasets/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path,header=True)

In [ ]:
# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The following parameters control how our model will behave in learning. 

**The batch_size parameter**  determine how much of the data is being used at a time to train the model. Ideally, you would want to use all the data in one run to train the model. However, doing so would result to a higher processor memory needed to run so we can process the data in batches which would be much easier for the processor.

**The learning_rate**  dictates how much we would adjust the model weight parameters. 

**The input_size** is defined by the image size (in this case a 28x28 pixel image). 

**The num_classes ** tells how many labels are there. In this case, 26, one for each letter.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = MedianStoppingPolicy()
hd_config = HyperDriveConfig(estimator=est,
  hyperparameter_sampling=param_sampling,
  policy=early_termination_policy,
  primary_metric_name='Accuracy',
  primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
  max_total_runs=16,
  max_concurrent_runs=4)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
         '--learning_rate': choice([1e-5,1e-4,1e-3,1e-2,1e-1]),
         '--batch_size': choice([64,128,256]), 
       #  '--epochs': choice([5,10,50]),
       #  '--dropout': choice([0.5,0.8,1])})

#TODO: Create your estimator and hyperdrive config
est = Estimator(source_directory='.',
                script_params=script_params,
                compute_target=cluster,
                entry_script='train_keras.py',
                pip_packages=['keras','tensorflow'])

hyperdrive_run_config = hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = param_sampling,
                                   primary_metric_name = 'Accuracy',
                                   primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs = 100,
                                   max_concurrent_runs = 4,
                                   policy = policy,
                                   estimator = est)

In [ ]:
#TODO: Submit your experiment

In [ ]:
run = exp.submit(est)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service